In [2]:
# import pandas as pd
# import os
# import glob
# import concurrent.futures
# import time

# directory = 'C:\\Users\\Trieu Pham\\Desktop\\230105 Payment Summary - 2023-01-06T085450.177'
# directory2 = 'C:\\Users\\trieu.pham\\Desktop\\230105 Payment Summary - 2023-01-06T085450.177'
# cate_dir = 'C:\\Users\\Trieu Pham\\Desktop\\DataCate.xlsx'
# cate_dir2 = 'C:\\Users\\trieu.pham\\Desktop\\DataCate.xlsx'

# # list of Excel files to read data from
# if os.path.exists(os.path.join(directory, '230101 Payment Summary - 2023-01-03T093340.600.xlsx')):
#     excel_files = glob.glob(os.path.join(directory, '*.xlsx'))
# else:
#     excel_files = glob.glob(os.path.join(directory2, '*.xlsx'))
# try:
#     # use cols 0, Barcode; 2, Category Name
#     # df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,2])
#     # use cols 0, Barcode; 3, SubCategory Name
#     df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,3])
#     pass
# except FileNotFoundError:
#     # df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,2])
#     # use cols 0, Barcode; 3, SubCategory Name
#     df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,3])
#     pass
# # create an empty list to store the dataframes
# # dfs = [] sho

# outputfile =  'dataT1.csv'
# # def excel2csv
# def excel2csv(inputfile, outputfile):
    
#     # Check if exist file_path
#     if not os.path.exists(outputfile):
#         with open(outputfile, 'w') as f:
#             pass

#     # Read the first line of the file
#     with open(outputfile, 'r') as f:
#         first_line = f.readline().strip()
#     # Check if the first line contains the expected column names
#     has_headers = 'InvoiceID' in first_line
#     df = pd.read_excel(inputfile, sheet_name='Sheet2', skiprows=2, usecols=[4,7,8,14])
#     # df2 Category
    
#     # merge with Category, on "Barcode"
#     df = df.merge(df2, on='Barcode', how='left')
    
#     # Check if the first line contains the expected column names
#     df.to_csv(outputfile, mode='a', header= not has_headers, index=False)


# # loop through the list of Excel files
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     [executor.submit(excel2csv, file, outputfile) for file in excel_files]
#     # for file in excel_files:
#     #     executor.submit(excel2csv,file)

In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Sample data
dtypes = {'InvoiceID': str, 'Barcode': str, 'Sum of Qty': str, 'Total Exclude VAT': str, 'SubCategory Name': str}
df1 = pd.read_csv('dataT1.csv', dtype=dtypes)
print(df1.head(10))
df1.info()


            InvoiceID        Barcode Sum of Qty Total Exclude VAT   
0  VN0001010201230001  8809005029823          1             24545  \
1  VN0001010201230002  8850157000864          1              8182   
2  VN0001010201230002  8850157511681          1             18182   
3  VN0001010201230002       93682961          1             21818   
4  VN0001010201230002  8934803043075          1              6364   
5  VN0001010201230003  8934804025766          2             18182   
6  VN0001010201230004  8936094291005          3             62727   
7  VN0001010201230005  8885019060120          1             16364   
8  VN0001010201230005  8801048951000          1             68182   
9  VN0001010201230006  8996001303764          1             72727   

     SubCategory Name  
0           Sugar CSD  
1         Meat snacks  
2  Alternative Snacks  
3      Chocolate Pack  
4        Salty Snacks  
5   Flavored Milk Sub  
6     Import Beer Can  
7           Sugar CSD  
8   Distilled Spirits  


In [4]:
bill2 = df1[df1.groupby('InvoiceID')['Sum of Qty'].transform('size') > 1]
bill2


,InvoiceID,Barcode,Sum of Qty,Total Exclude VAT,SubCategory Name
1,VN0001010201230002,8850157000864,1,8182,Meat snacks
2,VN0001010201230002,8850157511681,1,18182,Alternative Snacks
3,VN0001010201230002,93682961,1,21818,Chocolate Pack
4,VN0001010201230002,8934803043075,1,6364,Salty Snacks
7,VN0001010201230005,8885019060120,1,16364,Sugar CSD
...,...,...,...,...,...
14729759,VN0239022503230498,8850157100526,1,18182,OSC Juice
14729760,VN0239022503230498,8857099003274,1,10909,WIC/RIC Juice
14729761,VN0239022503230498,8938512932026,1,10000,Dessert Sub
14729762,VN0239022503230499,2010101000005,1,2727,Cup Of Ice


In [5]:
# 1.     Divide Bill 1 Items and Bill 2+ Items to get 3 things:
# a.      How many Unique Items are in Bill 2+
# b.      How many Quantity Items in Bill 2+
# c.      How much Net Sales Exclude VAT in Bill 2+


# a.      How many Unique Items are in Bill 2+
unique_items = bill2.drop_duplicates(subset='InvoiceID')

unique_items


,InvoiceID,Barcode,Sum of Qty,Total Exclude VAT,SubCategory Name
1,VN0001010201230002,8850157000864,1,8182,Meat snacks
7,VN0001010201230005,8885019060120,1,16364,Sugar CSD
9,VN0001010201230006,8996001303764,1,72727,Other cookies
13,VN0001010201230007,8934822901332,3,40908,Import Beer Can
15,VN0001010201230008,2010902000198,1,14545,Onigiri
...,...,...,...,...,...
14729744,VN0239022503230492,2701020000015,1,16364,Onigiri
14729750,VN0239022503230494,8935049501503,1,12727,Sugar CSD
14729752,VN0239022503230495,89686043686,2,14545,Instant Noodles Pack
14729756,VN0239022503230498,8936011773416,1,23636,Other Ice Cream


In [6]:
# b.      How many Quantity Items in Bill 2+

qty_item_arr = bill2['Sum of Qty'].to_list()
qty_item_arr = [int(x) for x in qty_item_arr if x != 'Sum of Qty']
sum_qty = sum(qty_item_arr)

sum_qty


14446588

In [7]:
# c.      How much Net Sales Exclude VAT in Bill 2+
qty_net_arr = bill2['Total Exclude VAT'].to_list()
qty_net_arr = [int(float(x)) for x in qty_net_arr if x != 'Total Exclude VAT']
sum_net = sum(qty_net_arr)

sum_net


257019046460

In [8]:
print(f"Number unique items: {unique_items['Barcode'].count()}")
print(f"Summary quantity items: {sum_qty}")
print(f"Summary net Sales Exclude VAT: {sum_net}")



Number unique items: 3682814
Summary quantity items: 14446588
Summary net Sales Exclude VAT: 257019046460


In [9]:
# 2.     Calculations:
# a.      What is the Percentage between Bill 1 and Bill 2+ in Total Net Sales, Total Bills, and Total Items => This only needs one result for each by month
# b.      How much Average Net Sales on Bill 2+ => To know the Real Basket Value
# c.      How many Average Unique Items in Bill 2+ => To know more insights the Basket
# d.      How many Average Items in Bill 2+ => To know more insights into the Basket


In [10]:
# Get Bill1
bill1 = df1[df1.groupby('InvoiceID')['Sum of Qty'].transform('size') == 1]

bill1

,InvoiceID,Barcode,Sum of Qty,Total Exclude VAT,SubCategory Name
0,VN0001010201230001,8809005029823,1,24545,Sugar CSD
5,VN0001010201230003,8934804025766,2,18182,Flavored Milk Sub
6,VN0001010201230004,8936094291005,3,62727,Import Beer Can
25,VN0001010201230011,8936011775724,1,10000,Stick
45,VN0001010201230018,8934614031216,1,6364,Soy milk
...,...,...,...,...,...
14729719,VN0239022503230483,260202000017,1,27273,Korean cuisine Sub
14729749,VN0239022503230493,8935049500155,1,10909,No/Less Sugar CSD
14729754,VN0239022503230496,2010805000356,1,17273,Other Cuisine Sub
14729755,VN0239022503230497,8935049501039,1,16364,Sugar CSD


In [19]:
# a.      What is the Percentage between Bill 1 and Bill 2+ in Total Net Sales, Total Bills, and Total Items 
# => This only needs one result for each by month

## Get bill1 Total Net Sales
qty_net_arr1 = bill1['Total Exclude VAT'].to_list()
qty_net_arr1 = [int(float(x)) for x in qty_net_arr1 if x != 'Total Exclude VAT']
sum_net1 = sum(qty_net_arr1)
## Percentage between Bill 1 and Bill 2+ in Total Net Sales
total_net_perc = sum_net1/sum_net

total_net_perc

0.2694741290925261

In [31]:
## Get bill1 Total Bills
import numpy as np
unique_bill1 = np.unique(bill1['InvoiceID'])
unique_bill2 = np.unique(bill2['InvoiceID'])
## Percentage between Bill 1 and Bill 2+ in Total Bills
total_bills_perc = len(unique_bill1)/len(unique_bill2)

total_bills_perc

0.7853934518550217

In [24]:
# Quantity items in bill1
qty_item_arr1 = bill1['Sum of Qty'].to_list()
qty_item_arr1 = [int(x) for x in qty_item_arr1 if x != 'Sum of Qty']
sum_qty1 = sum(qty_item_arr1)
## Percentage between Bill 1 and Bill 2+ in Total Items
total_qty_perc = sum_qty1/sum_qty

total_qty_perc

0.25142518081085996

In [32]:
# b.      How much Average Net Sales on Bill 2+ => To know the Real Basket Value

avg_net_sales = sum_net/len(unique_bill2)

avg_net_sales

69788.76654102

In [ ]:
# c.      How many Average Unique Items in Bill 2+ => To know more insights the Basket


In [12]:
# Calculate frequent itemsets using Apriori algorithm

frequent_itemsets = apriori(transactions, min_support=0.005, use_colnames=True)

frequent_itemsets

NameError: name 'transactions' is not defined

In [ ]:
# Generate association rules from the frequent itemsets

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bento),(Onigiri),0.036771,0.103501,0.006146,0.167143,1.614891,0.002340,1.076414
1,(Onigiri),(Bento),0.103501,0.036771,0.006146,0.059381,1.614891,0.002340,1.024037
2,(Sugar Tea),(Bento),0.156438,0.036771,0.006166,0.039415,1.071925,0.000414,1.002753
3,(Bento),(Sugar Tea),0.036771,0.156438,0.006166,0.167689,1.071925,0.000414,1.013519
4,(Cup Of Ice),(Energy Drinks Sub),0.083932,0.080919,0.021213,0.252744,3.123441,0.014422,1.229942
...,...,...,...,...,...,...,...,...,...
111,(Sandwich),(Sugar Tea),0.039010,0.156438,0.005125,0.131382,0.839839,-0.000977,0.971155
112,(Sugar CSD),(Sugar Tea),0.072158,0.156438,0.009103,0.126148,0.806377,-0.002186,0.965338
113,(Sugar Tea),(Sugar CSD),0.156438,0.072158,0.009103,0.058187,0.806377,-0.002186,0.985165
114,(Sugar Tea),(WIC/RIC Juice),0.156438,0.037814,0.009855,0.062998,1.666020,0.003940,1.026878


In [ ]:
# Display the association rules with support, lift, and confidence
import time
print(rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']])
rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']].to_csv('result'+ time.localtime() + '.csv', index=False)

NameError: name 'rules' is not defined